In [2]:
from __future__ import print_function

from IPython import display

import math
import matplotlib
import sklearn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from numpy import radians, cos, sin, arcsin, arccos, sqrt, pi, arctan2, degrees, arctan
import itertools
from datetime import datetime
from scipy import signal,ndimage, misc, stats
 
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()
tqdm.pandas(tqdm_notebook)

import osrm
from joblib import dump, load

from natsort import natsorted

from xgboost import XGBRegressor

pd.options.display.max_rows = 10

In [56]:
def haversine(lat1, lon1, lat2, lon2):
    #ensure using numpy and not math, or pandas series cannot be passed
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a))
    r = 6378.137 ##radius of earth km
    return c * r

def compute_dist(df):
    next_df = df.shift(1)
    dist = haversine(df.iloc[:,0], df.iloc[:,1],
                    next_df.iloc[:,0], next_df.iloc[:,1])
    return dist

def compute_time(df):
    next_df = df.shift(1)
#     df["time"] = pd.to_datetime(df["time_utc"], format="%Y-%m-%d %H:%M:%S.%f", errors='raise')
    timedelt = df["time"] - next_df["time"]
    return timedelt

def compute_speed(df):
    kinematics = df.copy()
    kinematics["distance_travelled"] = compute_dist(kinematics[["latitude", "longitude"]].astype(float)).values
    kinematics["time_elapsed"] = compute_time(kinematics).values
    kinematics["time_elapsed_seconds"] = kinematics["time_elapsed"]/np.timedelta64(1,'s')
    kinematics["speed m/s"] = (kinematics["distance_travelled"]*1000)/kinematics["time_elapsed_seconds"]
    kinematics["speed kmh"] = kinematics["speed m/s"]*3.6
    kinematics["bearing"] = compute_bearing(kinematics[["latitude", "longitude"]].astype(float))
    kinematics['bearing_diff'] = compute_bearing_diff(kinematics)
    kinematics["rate_of_turn"] = kinematics["bearing_diff"]/kinematics["time_elapsed_seconds"]
    kinematics["acceleration"] = (speed_diff(kinematics["speed m/s"])) /kinematics["time_elapsed_seconds"]
    kinematics.drop(columns = ['time_elapsed'], inplace = True)
    kinematics.fillna(0, inplace = True)
#     df["distance_travelled"] = kinematics["distance_travelled"].values
#     df["speed kmh"] = kinematics["speed kmh"].values
    return kinematics

def cal_bearing(lat1, lon1, lat2, lon2):
    """
    Calculates the bearing between two points using the formula
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    """
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1

    x = sin(dlon) * cos(lat2)
    y1 = cos(lat1) * sin(lat2)
    y2 = sin(lat1) * cos(lat2) * cos(dlon)
    y = y1 - y2

    initial_bearing = arctan2(x, y)

    initial_bearing = degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing


def compute_bearing(df):
    next_df = df.shift(1)
    bear = cal_bearing(df.iloc[:,0], df.iloc[:,1],
                    next_df.iloc[:,0], next_df.iloc[:,1])
    return bear


def speed_diff(df):
    next_df = df.shift(1)
    diff = df - next_df
    return diff


def bearing_diff(bearing, prev_bearing):
    
    """
    Calculates the absolute difference between two angles
    Parameters
      bearing: bearing in degrees of the first angle
      prev_bearing: bearing in degrees of the second angle
    Returns the difference in degrees as a float
    """

    # if bearing - prev_bearing <=180 then taking the absolute difference is correct
    if bearing - prev_bearing <= 180:
        bearing_diff = abs(bearing - prev_bearing)
    # otherwise if bearing is larger than previous_bearing the total difference is the prev_bearing plus the difference
    # between 360 and the bearing
    elif bearing > prev_bearing:
        bearing_diff = prev_bearing + (360 - bearing)
    # otherwise if prev_bearing is larger than bearing the total difference is the bearing plus the difference 
    # between 360 and the prev_bearing
    elif prev_bearing > bearing:
        bearing_diff = bearing + (360 - prev_bearing)
    # in all other cases take the absolute difference
    else:
        bearing_diff = abs(bearing - prev_bearing)

    return bearing_diff

def compute_bearing_diff(df):
    df1 = df.copy()
    df1["prev_bearing"] = df1["bearing"].shift(1)
    df1["bearing diff"] = df1.apply(lambda row: bearing_diff(row['bearing'],row['prev_bearing']),axis=1)
    return df1["bearing diff"]


In [277]:
chunk = 0
taxi = []
df =  pd.read_csv('/mnt/hgfs/FYP/porto_cleaned_100000.csv', parse_dates = ['time'])

In [282]:
df['time']

0         2013-07-01 00:00:58
1         2013-07-01 00:01:28
2         2013-07-01 00:01:43
3         2013-07-01 00:01:58
4         2013-07-01 00:02:13
                  ...        
4163115   2013-07-21 18:59:42
4163116   2013-07-21 18:59:57
4163117   2013-07-21 19:00:12
4163118   2013-07-21 19:00:27
4163119   2013-07-21 19:01:12
Name: time, Length: 4163120, dtype: datetime64[ns]

In [279]:
grouper = df.groupby('ID')

In [ ]:
df.groupby('ID').groups

In [225]:
import time

In [ ]:
taxi2 = []
counter = 0
for name, val in tqdm(grouper):
    idx_grper = val.groupby(val['Unnamed: 0'].diff().ne(1).cumsum()) #cause of the repeating indexes
    for name2, taxi in tqdm(idx_grper):
        taxi = compute_speed(taxi)
        i = 2
        i = str(i)
        
        summ = taxi.rolling(i + 'T', on = 'time').sum().add_suffix('_sum'+ i)
        summ.drop(summ.columns[[0,1,2,3,4,6,7,8,9,10,11,12]], axis =1, inplace = True)
        
        avg = taxi.rolling(i + 'T', on = 'time').mean().add_suffix('_mean'+ i)
        avg.drop(avg.columns[[0,1,4,6]], axis =1, inplace = True)
        
        stdd = taxi.rolling(i + 'T', on = 'time').std().add_suffix('_stddev' + i)
        stdd.drop(stdd.columns[[0,1,4,6]], axis = 1, inplace = True)
        
        kurt = taxi.rolling(i + 'T', on = 'time').kurt().add_suffix('_kurt' + i)
        kurt.drop(kurt.columns[[0,1,2,3,4,6]], axis = 1, inplace = True)
        
        skew = taxi.rolling(i + 'T', on = 'time').skew().add_suffix('_skew' + i)
        skew.drop(skew.columns[[0,1,2,3,4,6]], axis = 1, inplace = True)
        
        e = time.time()
        out = pd.concat([taxi, summ,avg, stdd, kurt, skew], axis = 1)
        prod = []
        
#         for index in list(itertools.combinations(out.index,2)):
#             prod.append(out.loc[index,:])

        for pair in itertools.combinations(out.values,2):
            pair = (pair[0], pair[1][1:4])
            prod.append(pair)
            
        foo = pd.DataFrame.from_records(prod)
        foo = pd.concat([pd.DataFrame(foo[0].values.tolist()), pd.DataFrame(foo[1].values.tolist())], axis = 1)
        
        foo.columns = out.columns.append(out.columns[1:4])
#         avg = taxi.resample(i+'S', on='time').mean().add_suffix('_mean' + i) #downsample the dataframe
        taxi2.append(foo)
    counter += 1
    if counter > 2:   
        break


 89%|████████▉ | 159/179 [00:29<00:01, 11.73it/s]

In [360]:
pd.concat(taxi2)

,Unnamed: 0,time,longitude,latitude,ID,distance_travelled,time_elapsed_seconds,speed m/s,speed kmh,bearing,...,distance_travelled_skew2,speed m/s_skew2,speed kmh_skew2,bearing_skew2,bearing_diff_skew2,rate_of_turn_skew2,acceleration_skew2,time,longitude,latitude
0,0,2013-07-01 07:06:43,-8.584353,41.163174,20000001,0.000000,0.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-01 07:06:58,-8.585289,41.162994
1,0,2013-07-01 07:06:43,-8.584353,41.163174,20000001,0.000000,0.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-01 07:07:13,-8.587512,41.163678
2,0,2013-07-01 07:06:43,-8.584353,41.163174,20000001,0.000000,0.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-01 07:07:43,-8.589024,41.164155
3,0,2013-07-01 07:06:43,-8.584353,41.163174,20000001,0.000000,0.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-01 07:07:58,-8.589024,41.164146
4,0,2013-07-01 07:06:43,-8.584353,41.163174,20000001,0.000000,0.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-01 07:08:13,-8.589060,41.164155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,12,2013-07-21 05:29:33,-8.612892,41.146020,20000003,0.107249,15.0,7.149908,25.739669,267.323271,...,-0.338014,-0.338014,-0.338014,-2.457934,2.031650,2.031650,0.453379,2013-07-21 05:30:03,-8.610867,41.145399
116,12,2013-07-21 05:29:33,-8.612892,41.146020,20000003,0.107249,15.0,7.149908,25.739669,267.323271,...,-0.338014,-0.338014,-0.338014,-2.457934,2.031650,2.031650,0.453379,2013-07-21 05:30:18,-8.610939,41.145345
117,13,2013-07-21 05:29:48,-8.610849,41.146056,20000003,0.171306,15.0,11.420426,41.113535,268.660186,...,-0.899894,-0.899894,-0.899894,-2.468645,2.039084,2.039084,0.799394,2013-07-21 05:30:03,-8.610867,41.145399
118,13,2013-07-21 05:29:48,-8.610849,41.146056,20000003,0.171306,15.0,11.420426,41.113535,268.660186,...,-0.899894,-0.899894,-0.899894,-2.468645,2.039084,2.039084,0.799394,2013-07-21 05:30:18,-8.610939,41.145345


In [284]:
def foobar(df):
#     print(df)
    pd.DataFrame(df).hist()
    return df.sum()

In [ ]:
taxi.set_index('time')['speed kmh'].rolling('5T').apply(foobar)

In [160]:
kurt

,Unnamed: 0_kurt2,time_kurt2,longitude_kurt2,latitude_kurt2,ID_kurt2,distance_travelled_kurt2,time_elapsed_seconds_kurt2,speed m/s_kurt2,speed kmh_kurt2,bearing_kurt2,bearing_diff_kurt2,rate_of_turn_kurt2,acceleration_kurt2
24648,NaN,2013-07-01 07:06:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24649,NaN,2013-07-01 07:06:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24650,NaN,2013-07-01 07:07:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24651,-1.2,2013-07-01 07:07:43,-3.237057,249903.085637,NaN,-0.463854,1.5,1.849328,1.849328,1.590288,3.996530,3.999140,2.066912
24652,-1.2,2013-07-01 07:07:58,-2.859517,-110973.901868,NaN,-1.698627,2.0,1.350815,1.350815,-3.086571,2.795599,2.789173,1.607175
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24661,-1.2,2013-07-01 07:10:13,-0.944093,443.253003,NaN,-1.750594,NaN,-1.750594,-1.750594,0.428719,0.518179,0.518179,2.296937
24662,-1.2,2013-07-01 07:10:28,-0.968403,-3026.235248,NaN,-0.335462,NaN,-0.335462,-0.335462,-0.178179,0.518506,0.518506,2.390615
24663,-1.2,2013-07-01 07:10:43,-1.197516,-11166.286109,NaN,3.106925,NaN,3.106925,3.106925,-2.187849,5.043874,5.043874,2.207283
24664,-1.2,2013-07-01 07:10:58,-1.720617,-80050.664280,NaN,4.170093,NaN,4.170093,4.170093,-0.139373,6.022093,6.022093,1.201893


In [115]:
df = pd.DataFrame({'B': [0, 1, 2, 30, 4]},
                  index = [pd.Timestamp('20130101 09:00:00'),
                           pd.Timestamp('20130101 09:10:02'),
                           pd.Timestamp('20130101 09:20:03'),
                           pd.Timestamp('20130101 09:30:05'),
                           pd.Timestamp('20130101 09:40:06')])

In [116]:
df.rolling('20T').mean()

,B
2013-01-01 09:00:00,0.0
2013-01-01 09:10:02,0.5
2013-01-01 09:20:03,1.5
2013-01-01 09:30:05,16.0
2013-01-01 09:40:06,17.0
